In [ ]:
#!pip install category_encoders

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.api.types import is_object_dtype, is_numeric_dtype, is_bool_dtype

import sklearn
sklearn.set_config(transform_output="pandas")
import category_encoders as ce

from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.metrics import accuracy_score, cohen_kappa_score

from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
import xgboost as xgb

import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv")
sub = pd.read_csv("/kaggle/input/playground-series-s4e11/sample_submission.csv")

In [ ]:
# This variable allows to run or not various pre-processing techniques
DO = "norm_7"

In [ ]:
LABEL = "Depression"
CAT_COLUMNS = df_train.select_dtypes(include=['object']).columns.tolist()
NUM_COLUMNS = [c for c in df_train.columns if c not in CAT_COLUMNS + ["id", LABEL]]
FEATURES = CAT_COLUMNS +  NUM_COLUMNS

SPLITS = 20

# 30 SPLITS

In [ ]:
"""
SPLIT :  1
Score CB: 0.9149253731343283
Score LGB 0.929637526652452
Score XGB 0.9373134328358209
Score HGB 0.9383795309168443
Score GB 0.9402985074626866
Score RF 0.9353944562899786
Score LR 0.9353944562899786
Score final 30 splits  :  0.9330490405117271
Score from vals 30 splits  :  0.9353944562899786


"""

# 10 SPLITS - Results I gor for various versions

In [ ]:
"""
Version 38
SPLIT :  1
Score XGB 0.9414356787491116
Score HGB 0.9398009950248756
Score LR 0.9393745557924662
Score final 10 splits  :  0.9402037431888178

Version 39
SPLIT :  1
Score CB: 0.9243781094527364
Score LGB 0.9350390902629708
Score XGB 0.9414356787491116
Score HGB 0.9394456289978678
Score LR 0.9393745557924662
Score final 10 splits  :  0.9359346126510306
"""

# 20 SPLITS

In [ ]:
"""
Version 55
SPLIT :  1
Score XGB 0.9448471926083867
Score HGB 0.9444207533759773
Score GB 0.9439943141435678
Score final 20 splits  :  0.9444207533759773
Score from vals 20 splits  :  0.9449893390191898

Version 56
SPLIT :  1
Score XGB 0.9448471926083867
Score HGB 0.9444207533759773
Score GB 0.9439943141435678
Score LR 0.9424307036247335
Score final 20 splits  :  0.9439232409381664
Score from vals 20 splits  :  0.9448471926083867

Version 54
SPLIT :  1
Score XGB 0.9448471926083867
Score HGB 0.9422885572139303
Score GB 0.9421464108031272
Score final 20 splits  :  0.9430940535418147

Version 37
Score XGB 0.9448471926083867
Score HGB 0.9417199715707179
Score LR 0.940724946695096
Score final 20 splits  :  0.9424307036247335

Version 52
SPLIT :  1
Score XGB 0.9448471926083867
Score HGB 0.9431414356787491
Score GB 0.9402985074626866
Score RF 0.942999289267946
Score LR 0.940724946695096
Score final 20 splits  :  0.9424022743425728



Version 49
SPLIT :  1
Score CB: 0.9279317697228145
Score LGB 0.9395877754086709
Score XGB 0.9448471926083867
Score HGB 0.9425728500355366
Score GB 0.9417199715707179
Score RF 0.941862117981521
Score LR 0.940724946695096
Score final 20 splits  :  0.939892374860392

Version 47
SPLIT :  1
Score CB: 0.9279317697228145
Score LGB 0.9395877754086709
Score XGB 0.9448471926083867
Score HGB 0.9437100213219616
Score LR 0.940724946695096
Score final 20 splits  :  0.939360341151386

Version 40
SPLIT :  1
Score CB: 0.9279317697228145
Score LGB 0.9395877754086709
Score XGB 0.9448471926083867
Score HGB 0.9425728500355366
Score LR 0.940724946695096
Score final 20 splits  :  0.939132906894101




SPLIT :  1
Score CB: 0.9279317697228145
Score LGB 0.9395877754086709
Score XGB 0.9448471926083867
Score HGB 0.941862117981521
Score LR 0.9198294243070363 ( + class weight)
Score final 20 splits  :  0.9348116560056858
"""

In [ ]:
colx = df_train["Name"].unique()
colv = df_test["Name"].unique()

colx_vc = df_train["Name"].value_counts()
colv_vc = df_test["Name"].value_counts()

col_name_v = [c for c in colv if c not in colx]
col_name_x = [c for c in colx if c not in colv]

def parse_degree(df, train_df=True):
    COLUMN = "Degree"
    cols = ['Mechanical Engineer', 'Travel Consultant', 'Magan', 'Doctor', 'Nalini', 'Degree', 'Badhya', 'HR Manager', 'Bhavesh', 'Business Analyst', '0', 'Mthanya', 'Working Professional', 'UX/UI Designer', 'Esha', 'Plumber', '24', '29', '8.95', 'Aadhya', '20', 'Vivaan', 'Advait', 'B.Study_Hours', 'Vibha','B. Gender','Gagan','5.65','Eshita','Navya','Pune', 'Bian','Kavya','Vrinda', 'M', 'E.Ed', '3.0','Moham','Rupak','Banchal', 'X/UI Designer', 'Veda', 'Bhopal', 'Kalyan', 'Unite', '5.61', 'Brit', 'B.03', 'Ritik', '5.56', '7.06', 'Brithika', 'Pihu', 'Jhanvi', 'Aarav', 'Entrepreneur', '8.56', 'Mahika', 'Lata', 'S.Arch', 'Marsh','5.88', 'B.3.79', 'Mihir', 'Data Scientist']
    
    col_vc = df[COLUMN].value_counts()
    one_vals = [i for i in col_vc.index if col_vc[i] < 100]

    if DO in ["norm_6", "norm_7"]:
        df.loc[df[COLUMN].isin(cols), COLUMN] = "FAKE"
        df.loc[df[COLUMN].isin(one_vals), COLUMN] = "RARE"
    elif DO in ["norm_4", "norm_5"]:
        df.loc[df[COLUMN].isin(cols+one_vals), COLUMN] = np.nan
    else:
        if train_df:
            df = df.loc[~df[COLUMN].isin(cols+one_vals)]
        else:
            df.loc[df[COLUMN].isin(cols+one_vals), COLUMN] = np.nan
            
    return df    
        

def parse_dietary_habits(df, train_df=True):
    COLUMN = "Dietary Habits"
    cols = ['Yes', 'No', 'MCA', 'Resistant', 'Pratham', 'Gender', '3', '1.0', 'Mihir', 'Vegas', 'Electrician', 'M.Tech', 'Class 12', 'Male', '2', 'Indoor', 'Prachi', 'Hormonal', 'Academic', 'Educational', 'Soham', 'Vivaan', 'Raghav', 'Naina', 'Kolkata', 'BSc']
    
    col_vc = df[COLUMN].value_counts()
    one_vals = [i for i in col_vc.index if col_vc[i] < 100]
    
    if DO in ["norm_6", "norm_7"]:
        df.loc[df[COLUMN].isin(cols), COLUMN] = -10
        df.loc[df[COLUMN].isin(one_vals), COLUMN] = -20
    elif DO in ["norm_4", "norm_5"]:
        df.loc[df[COLUMN].isin(cols+one_vals), COLUMN] = np.nan
    else:
        if train_df:
            df = df.loc[~df[COLUMN].isin(cols)]    
        else:
            df.loc[df["Dietary Habits"].isin(cols+one_vals), COLUMN] = np.nan
            
    df.loc[df[COLUMN]=='5 Healthy', COLUMN] = "Healthy"
    df.loc[df[COLUMN]=='5 Unhealthy', COLUMN] = "Unhealthy"
      
    df.loc[df[COLUMN]=='Moderate', COLUMN] = 3
    df.loc[df[COLUMN]=='Healthy', COLUMN] = 4
    df.loc[df[COLUMN]=='Unhealthy', COLUMN] = 1
    df.loc[df[COLUMN]=='Less than Healthy', COLUMN] = 2
    df.loc[df[COLUMN]=='Less Healthy', COLUMN] = 2
    df.loc[df[COLUMN]=='No Healthy', COLUMN] = 1
    df.loc[df[COLUMN]=='More Healthy', COLUMN] = 5
    df.loc[df[COLUMN]=='Mealy', COLUMN] = 3
    
    df[COLUMN] = df[COLUMN].astype(np.float32)
    
    return df


def parse_profession(df, train_df=True):
    COLUMN = "Profession"
    cols_profession_to_delete = ['PhD','MBBS', 'B.Ed','M.Ed','BBA','MBA', 'LLM','BCA','B.Com', 'BE','Simran', '3M', 'Name', 'No', '24th', 'Unveil', 'Unhealthy', 'Yuvraj', 'Yogesh', 'Patna', 'Nagpur', 'Pranav', 'Visakhapatnam', 'Moderate', 'Manvi', 'Yogesh', 'Samar', 'Surat']
         
    col_vc = df[COLUMN].value_counts()
    one_vals = [i for i in col_vc.index if col_vc[i] < 100]

    if DO in ["norm_6", "norm_7"]:
        df.loc[df[COLUMN].isin(cols_profession_to_delete), COLUMN] = "FAKE"
        df.loc[df[COLUMN].isin(one_vals), COLUMN] = "RARE"
    elif DO in ["norm_4", "norm_5"]:
        df.loc[df[COLUMN].isin(cols_profession_to_delete+one_vals), COLUMN] = np.nan
    else:    
        if train_df:
            df = df.loc[~df[COLUMN].isin(cols_profession_to_delete+one_vals)]
        else:
            df.loc[df[COLUMN].isin(cols_profession_to_delete+one_vals), COLUMN] = np.nan
        
    return df


def parse_city(df, train_df=True):
    COLUMN = "City"
    cols_city_to_delete =  ['3.0', 'City', 'Less than 5 Kalyan', 'No', 'No.12', 'M.Tech', 'MCA', 'ME', 'M.Com']
    
    col_vc = df[COLUMN].value_counts()
    one_vals = [i for i in col_vc.index if col_vc[i] < 100]

    if DO in ["norm_6", "norm_7"]:
        df.loc[df[COLUMN].isin(cols_city_to_delete), COLUMN] = "FAKE"
        df.loc[df[COLUMN].isin(one_vals), COLUMN] = "RARE"
    elif DO in ["norm_4", "norm_5"]:
        df.loc[df[COLUMN].isin(cols_city_to_delete+one_vals), COLUMN] = np.nan
    else:    
        if train_df:
            df = df.loc[~df[COLUMN].isin(cols_city_to_delete+one_vals)]
        else:
            df.loc[df[COLUMN].isin(cols_city_to_delete+one_vals), COLUMN] = np.nan
        
    return df
    
def parse_sleeping_duration(df, train_df=True):
    COLUMN = "Sleep Duration"
    cols_sleping_duration_to_delete = ["No", "Vivian", "Vivan", "Meerut", "Have_you_ever_had_suicidal_thoughts", "Sleep_Duration", "Work_Study_Hours", "Pune", "Indore"]
    
    col_vc = df[COLUMN].value_counts()
    one_vals = [i for i in col_vc.index if col_vc[i] < 100]

    if DO in ["norm_6", "norm_7"]:
        df.loc[df[COLUMN].isin(cols_sleping_duration_to_delete), COLUMN] = -10
        df.loc[df[COLUMN].isin(one_vals), COLUMN] = -20
    elif DO in ["norm_4", "norm_5"]:
        df.loc[df[COLUMN].isin(cols_sleping_duration_to_delete+one_vals), COLUMN] = np.nan
    else:    
        if train_df:
            df = df.loc[~df[COLUMN].isin(cols_sleping_duration_to_delete+one_vals)]
        else:
            df.loc[df[COLUMN].isin(cols_sleping_duration_to_delete+one_vals), COLUMN] = np.nan
            
    df.loc[df[COLUMN]=='60-65 hours', COLUMN] = 63
    df.loc[df[COLUMN]=='8-89 hours', COLUMN] = 85
    df.loc[df[COLUMN]=='20-21 hours', COLUMN] = 20.5
    df.loc[df[COLUMN]=='6 hours', COLUMN] = 6
    df.loc[df[COLUMN]=='50-75 hours', COLUMN] = 50
    df.loc[df[COLUMN]=='9-10 hours', COLUMN] = 9.5
    df.loc[df[COLUMN]=='5-6 hours', COLUMN] = 5.5
    df.loc[df[COLUMN]=='7-8 hours', COLUMN] = 7.5
    df.loc[df[COLUMN]=='1-2 hours', COLUMN] = 1.5
    df.loc[df[COLUMN]=='4-6 hours', COLUMN] = 4.5
    df.loc[df[COLUMN]=='6-7 hours', COLUMN] = 6.5
    df.loc[df[COLUMN]=='10-11 hours', COLUMN] = 10
    df.loc[df[COLUMN]=='8-9 hours', COLUMN] = 8.5
    df.loc[df[COLUMN]=='6-8 hours', COLUMN] = 7
    df.loc[df[COLUMN]=='3-4 hours', COLUMN] = 3.5
    df.loc[df[COLUMN]=='2-3 hours', COLUMN] = 2.5
    df.loc[df[COLUMN]=='40-45 hours', COLUMN] = 42.5
    df.loc[df[COLUMN]=='1-3 hours', COLUMN] = 2
    df.loc[df[COLUMN]=='9-11 hours', COLUMN] = 10
    df.loc[df[COLUMN]=='4-5 hours', COLUMN] = 4.5
    df.loc[df[COLUMN]=='55-66 hours', COLUMN] = 60.5
    df.loc[df[COLUMN]=='9-6 hours', COLUMN] = 7.5
    df.loc[df[COLUMN]=='1-6 hours', COLUMN] = 3.5
    df.loc[df[COLUMN]=='35-36 hours', COLUMN] = 35.5
    df.loc[df[COLUMN]=='45-48 hours', COLUMN] = 46.5
    df.loc[df[COLUMN]=='10-6 hours', COLUMN] = 8.1
    df.loc[df[COLUMN]=='8 hours', COLUMN] = 8
    df.loc[df[COLUMN]=='49 hours', COLUMN] = 49
    df.loc[df[COLUMN]=='3-6 hours', COLUMN] = 4.5
    df.loc[df[COLUMN]=='9-5 hours', COLUMN] = 7
    df.loc[df[COLUMN]=='9-5', COLUMN] = 7
    df.loc[df[COLUMN]=='Unhealthy', COLUMN] = 1
    df.loc[df[COLUMN]=='No', COLUMN] = 0
    df.loc[df[COLUMN]=='than 5 hours', COLUMN] = 5
    df.loc[df[COLUMN]=='Less than 5 hours', COLUMN] = 4.9
    df.loc[df[COLUMN]=='More than 8 hours', COLUMN] = 8
    df.loc[df[COLUMN]=='Moderate', COLUMN] = 6
    df.loc[df[COLUMN]=='45', COLUMN] = 45
    df.loc[df[COLUMN]=='0', COLUMN] = 0
    df[COLUMN] = df[COLUMN].astype(np.float32)
    return df
    
def parse_cat(df, train_df=True):
    #if "Name" in df.columns:
    #    df.drop(columns=["Name"], inplace=True)
    df = parse_sleeping_duration(df, train_df)
    df = parse_city(df, train_df)
    df = parse_profession(df, train_df)
    df = parse_dietary_habits(df, train_df)
    df = parse_degree(df, train_df)
    return df

In [ ]:

if DO != "": 
    df_train = parse_cat(df_train)
    df_test = parse_cat(df_test, train_df=False)


In [ ]:
df_train["missing"] = df_train.isna().sum(axis=1)
df_test["missing"] = df_test.isna().sum(axis=1)

In [ ]:
CAT_COLUMNS = df_train.select_dtypes(include=['object']).columns.tolist()
NUM_COLUMNS = [c for c in df_train.columns if c not in CAT_COLUMNS + ["id", LABEL]]
FEATURES = CAT_COLUMNS +  NUM_COLUMNS

In [ ]:
if DO == "":
    COLS_TO_ENCODE = CAT_COLUMNS
else:    
    COLS_TO_ENCODE = ["Name", "Gender", "City", "Working Professional or Student", "Profession", "Degree", "Have you ever had suicidal thoughts ?", "Family History of Mental Illness"]

# Concat train and test in order not to have unknown values. 
# This might work for these mid-level competitions.
df_all = pd.concat([df_train[FEATURES], df_test[FEATURES]])

#oe = OrdinalEncoder(encoded_missing_value = -1, handle_unknown='use_encoded_value', unknown_value=-1)
oe = ce.TargetEncoder(COLS_TO_ENCODE)
oe.fit(df_train[COLS_TO_ENCODE], df_train[LABEL])
df_train[COLS_TO_ENCODE] = oe.transform(df_train[COLS_TO_ENCODE])
df_test[COLS_TO_ENCODE] = oe.transform(df_test[COLS_TO_ENCODE])


In [ ]:
df_train[COLS_TO_ENCODE] = df_train[COLS_TO_ENCODE].fillna("NAN")
df_test[COLS_TO_ENCODE] = df_test[COLS_TO_ENCODE].fillna("NAN")

In [ ]:
# A test I did but gave a poorer score
if DO == "norm_5":
    imp = IterativeImputer(estimator=HistGradientBoostingRegressor(), missing_values=-1)

    df_all = pd.concat([df_train[FEATURES], df_test[FEATURES]])
    imp.fit(df_all[COLS_TO_ENCODE])
    df_train[COLS_TO_ENCODE] = imp.transform(df_train[COLS_TO_ENCODE])
    df_train[COLS_TO_ENCODE] = df_train[COLS_TO_ENCODE].round()
    df_test[COLS_TO_ENCODE] = imp.transform(df_test[COLS_TO_ENCODE])
    df_test[COLS_TO_ENCODE] = df_test[COLS_TO_ENCODE].round()
        

In [ ]:
se = SimpleImputer()
se.fit(df_train[NUM_COLUMNS])
df_train[NUM_COLUMNS] = se.transform(df_train[NUM_COLUMNS])
df_test[NUM_COLUMNS] = se.transform(df_test[NUM_COLUMNS])

In [ ]:
se = RobustScaler()
se.fit(df_train[NUM_COLUMNS])

df_train_rs = df_train.copy()
df_test_rs = df_test.copy()

df_train_rs[NUM_COLUMNS] = se.transform(df_train[NUM_COLUMNS])
df_test_rs[NUM_COLUMNS] = se.transform(df_test[NUM_COLUMNS])

In [ ]:
#df_train.fillna(-1, inplace=True)
#df_test.fillna(-1, inplace=True)

In [ ]:
#df_test.to_csv("test_"+DO+".csv", index=False)
#df_train.to_csv("train_"+DO+".csv", index=False)

In [ ]:
#corr = df_train.corr()
#sns.heatmap(corr);

In [ ]:
X = df_train[FEATURES]
Y = df_train[LABEL]

X_rs = df_train_rs[FEATURES]
Y_rs = df_train_rs[LABEL]

In [ ]:
# Best parameters obtained with Optuna for various algorithms :

In [ ]:
# LightGBM
# 0.9395877754086709 - Splits 20
# 0.929637526652452 - Splits 30
params_lgb =  {'reg_lambda': 0.0026698283995182604, 'reg_alpha': 0.0989129184454625, 'colsample_bytree': 1.0, 'learning_rate': 0.060731608923579795, 'n_estimators': 3085, 'max_depth': 21, 'num_leaves': 41, 'min_split_gain ': 0.7936829445500752, 'boosting_type': 'gbdt', 'min_child_weight': 1, 'min_child_samples': 15, 'subsample': 0.4, 'class_weight': 'balanced', 'verbose': -1, 'objective': 'binary', 'random_state': 0}


# CatBoost
# 0.9279317697228145 - Splits 20
# 0.9149253731343283 - Splits 30
params_cb = {'reg_lambda': 0.8352770337753764, 'learning_rate': 0.02208182328039188, 'iterations': 3853, 'max_depth': 16, 'bagging_temperature': 21, 'grow_policy': 'Lossguide', 'max_leaves': 181, 'min_data_in_leaf': 37, 'early_stopping_rounds': 96, 'random_state': 0, 'auto_class_weights': 'Balanced', 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'logging_level': 'Silent'}


#XGBoost
# 0.9448471926083867 - Splits 20
# 0.9373134328358209 - Splits 30
params_xgb = {'reg_lambda': 1.5428511958840945, 'reg_alpha': 1.9784889391087677, 'colsample_bytree': 0.24941596115970358, 'colsample_bylevel': 0.7741734319140999, 'colsample_bynode': 0.48878500754238585, 'subsample': 0.8, 'grow_policy': 'lossguide', 'tree_method': 'hist', 'booster': 'gbtree', 'learning_rate': 0.7120749229294119, 'n_estimators': 4308, 'max_depth': 6, 'max_leaves': 35, 'random_state': 0, 'min_child_weight': 0.6958532789138221, 'num_parallel_tree': 7, 'importance_type': 'cover', 'early_stopping_rounds': 92, 'verbosity': None}


# LogisticRegression
# 0.9408670931058991 - Splits 20
# 0.9353944562899786 - Splits 30
params_lr = {'penalty': 'l2', 'C': 3.0004668024183463, 'max_iter': 4175, 'solver': 'newton-cg', 'random_state': 0, 'fit_intercept': True, 'verbose': 0}


#HistGradientBoosting
# 0.9444207533759773 - Splits 20
params_hgb = {'max_iter': 1520, 'l2_regularization': 8.191871920529406, 'learning_rate': 0.06118116610086637, 'max_depth': 36, 'random_state': 0, 'max_leaf_nodes': 78, 'early_stopping': 'auto', 'verbose': 0}
#{'max_iter': 2304, 'max_leaf_nodes': 17, 'l2_regularization': 7.995559326469298, 'learning_rate': 0.6463109974953717, 'max_depth': 25, 'random_state': 0, 'early_stopping': 'auto', 'verbose': 0}


# 0.9383795309168443 - Splits 30
params_hgb_30 = {'max_iter': 1344, 'l2_regularization': 0.8759140605723984, 'learning_rate': 0.0739467989453549, 'max_depth': 22, 'random_state': 0, 'max_leaf_nodes': 88, 'early_stopping': 'auto', 'verbose': 0}


#GradientBoosting
# 0.9439943141435678 - Splits 20
params_gb = {'n_estimators': 3333, 'subsample': 0.9698554748709222, 'min_samples_split': 0.6662565698253853, 'min_samples_leaf': 0.0007345948700016973, 'min_weight_fraction_leaf': 0.014786932404866664, 'learning_rate': 0.978339301630668, 'max_depth': 87, 'random_state': 0, 'max_leaf_nodes': 24, 'n_iter_no_change': 5, 'verbose': 0}

# 0.9402985074626866 - Splits 30
params_gb_30 = {'n_estimators': 4451, 'subsample': 0.8965541536141035, 'min_samples_split': 0.1903502832441657, 'min_samples_leaf': 0.00438540641029464, 'min_weight_fraction_leaf': 0.027896901188861886, 'learning_rate': 0.31155313567602394, 'max_depth': 57, 'random_state': 0, 'max_leaf_nodes': 66, 'n_iter_no_change': 27, 'verbose': 0}
    

In [ ]:
pred_cb = 0
pred_xgb = 0
pred_lgb = 0
pred_hgb = 0
pred_gb = 0
pred_rf = 0
pred_lr = 0

Run or not certain algorithms in order to merge predictions: 

In [ ]:
DO_CB = True
DO_LGB = True
DO_XGB = False
DO_HGB = True
DO_GB = False
DO_RF = False
DO_LR = True

In [ ]:
if DO in ["norm_7"]:
    df_folds = pd.read_csv("/kaggle/input/db-s4e11/tabular_s4e11_folds_"+str(SPLITS)+".csv")

    pred_val_cb = 0
    pred_val_xgb = 0
    pred_val_lgb = 0
    pred_val_hgb = 0
    pred_val_gb = 0
    pred_val_rf = 0
    pred_val_lr = 0
    
    count_algos = 0
    score_final = 0

    pred_val = 0
    
    for VAL in range(1, 2):
        print("SPLIT : ", VAL)
        id_val = df_folds.loc[df_folds["folds"]==VAL, "id"]
        id_train = df_folds.loc[df_folds["folds"]!=VAL, "id"]
        
        X_train = df_train.loc[df_train["id"].isin(id_train), FEATURES]
        X_val = df_train.loc[df_train["id"].isin(id_val), FEATURES]
        y_train = df_train.loc[df_train["id"].isin(id_train), LABEL]
        y_val = df_train.loc[df_train["id"].isin(id_val), LABEL]

        if DO_CB:
            model_cb = CatBoostClassifier(**params_cb)
            model_cb.fit(X_train, y_train, eval_set=[(X_val, y_val)])
            pred_cb += model_cb.predict_proba(df_test[FEATURES])
            pred_val_cb += model_cb.predict_proba(X_val)

            pred_val += pred_val_cb
    
            score_cb = accuracy_score(y_val, np.argmax(pred_val_cb, axis=1))
            print("Score CB:", score_cb)

            count_algos = count_algos + 1
            score_final += score_cb

        if DO_LGB:
            model = LGBMClassifier(**params_lgb)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
            pred_lgb += model.predict_proba(df_test[FEATURES])
            pred_val_lgb += model.predict_proba(X_val)

            pred_val += pred_val_lgb
            
            score_lgb = accuracy_score(y_val, np.argmax(pred_val_lgb, axis=1))
            print("Score LGB", score_lgb)

            count_algos = count_algos + 1
            score_final += score_lgb

        if DO_XGB:
            model_xgb = xgb.XGBClassifier(**params_xgb)
            model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            pred_xgb += model_xgb.predict_proba(df_test[FEATURES])
            pred_val_xgb += model_xgb.predict_proba(X_val)

            pred_val += pred_val_xgb
    
            score_xgb = accuracy_score(y_val, np.argmax(pred_val_xgb, axis=1))
            print("Score XGB", score_xgb)

            count_algos = count_algos + 1
            score_final += score_xgb

        if DO_HGB:
            if SPLITS == 30:
                params_hgb = params_hgb_30
                
            model_hgb = HistGradientBoostingClassifier(**params_hgb)
            model_hgb.fit(X_train, y_train)
            pred_hgb += model_hgb.predict_proba(df_test[FEATURES])
            pred_val_hgb += model_hgb.predict_proba(X_val)

            pred_val += pred_val_hgb
    
            score_hgb = accuracy_score(y_val, np.argmax(pred_val_hgb, axis=1))
            print("Score HGB", score_hgb)   

            count_algos = count_algos + 1
            score_final +=  score_hgb

        if DO_GB:
            if SPLITS == 30:
                params_gb = params_gb_30
                
            model_gb = GradientBoostingClassifier(**params_gb)
            model_gb.fit(X_train, y_train)
            pred_gb += model_gb.predict_proba(df_test[FEATURES])
            pred_val_gb += model_gb.predict_proba(X_val)

            pred_val += pred_val_gb
    
            score_gb = accuracy_score(y_val, np.argmax(pred_val_gb, axis=1))
            print("Score GB", score_gb)   

            count_algos = count_algos + 1
            score_final += score_gb

        if DO_RF:
            model_rf = RandomForestClassifier()
            model_rf.fit(X_train, y_train)
            pred_rf += model_rf.predict_proba(df_test[FEATURES])
            pred_val_rf += model_rf.predict_proba(X_val)

            pred_val += pred_val_rf
    
            score_rf = accuracy_score(y_val, np.argmax(pred_val_rf, axis=1))
            print("Score RF", score_rf)   

            count_algos = count_algos + 1
            score_final += score_rf
        
        if DO_LR:
            model_lr = LogisticRegression(**params_lr)
            model_lr.fit(X_train, y_train)
            pred_lr += model_lr.predict_proba(df_test[FEATURES])
            pred_val_lr += model_lr.predict_proba(X_val)

            pred_val += pred_val_lr
    
            score_lr = accuracy_score(y_val, np.argmax(pred_val_lr, axis=1))
            print("Score LR", score_lr)    

            count_algos = count_algos + 1
            score_final += score_lr

    
    score_final /= float(count_algos)
    
    print("Score final", SPLITS, "splits", " : ", score_final)
    
    score_from_vals = accuracy_score(y_val, np.argmax(pred_val, axis=1))
    print("Score from vals", SPLITS, "splits", " : ", score_from_vals)

In [ ]:

if DO not in ["norm_7"]:
    if DO in ["norm_5"]:
        model = CatBoostClassifier(**params_cb)
        model.fit(X_rs,Y_rs)
        pred_cb = model.predict_proba(df_test_rs[FEATURES])
    elif DO not in ["norm_7"]:
        model = CatBoostClassifier(logging_level="Silent")
        model.fit(X_rs,Y_rs)
        pred_cb = model.predict_proba(df_test_rs[FEATURES])
    
    if DO in ["norm_5"]:
        model = LGBMClassifier(**params_lgb)
    else:
        model = LGBMClassifier(verbose=-1)
    model.fit(X,Y)
    pred_lgb = model.predict_proba(df_test[FEATURES])
    
    model_xgb = xgb.XGBClassifier(verbose=-1)
    model_xgb.fit(X,Y)
    pred_xgb = model_xgb.predict_proba(df_test[FEATURES])


pred = pred_cb + pred_lgb + pred_xgb + pred_hgb + pred_gb + pred_lr + pred_val_rf

In [ ]:
sub[LABEL] = np.argmax(pred, axis=1)
sub.to_csv("submission.csv", index=False)